In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aerial-cactus-identification/sample_submission.csv
/kaggle/input/aerial-cactus-identification/train.zip
/kaggle/input/aerial-cactus-identification/test.zip
/kaggle/input/aerial-cactus-identification/train.csv


In [2]:
import pandas as pd

data_path = '/kaggle/input/aerial-cactus-identification/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [8]:
from zipfile import ZipFile

with ZipFile(data_path + 'train.zip') as zipper:
    zipper.extractall()
with ZipFile(data_path + 'test.zip') as zipper:
    zipper.extractall()

In [3]:
# 결과값 고정을 위한 것

import torch
import random
import numpy as np
import os

seed = 50
os.environ['PYTHONASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed) # CPU - seed
torch.cuda.manual_seed(seed) # GPU - seed
torch.cuda.manual_seed_all(seed) # MultiGPU - seed
torch.backends.cudnn.deterministic = True # 확정 연산
torch.backends.cudnn.benchmark = False # 벤치마크 해제
torch.backends.cudnn.enabled = False # cudnn 해제

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
device

device(type='cuda')

### 데이터 분리

In [11]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(labels, test_size = 0.1, stratify=labels['has_cactus'], random_state=50)


In [12]:
print(len(train), len(valid))

15750 1750


In [13]:
import cv2
from torch.utils.data import Dataset

In [31]:
class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    # idx에 해당하는 데이터 반환 매서드
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0] # 이미지 ID
        img_path = self.img_dir + img_id # 이미지 파일 경로
        image = cv2.imread(img_path) # 이미지 파일 읽기
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        label = self.df.iloc[idx, 1] # 이미지 레이블 (타깃값)
        
        if self.transform is not None:
            image = self.transform(image) # 변환기가 있다면 이미지 변환
        return image, label
        

In [32]:
from torchvision import transforms

transform = transforms.ToTensor()

In [33]:
dataset_train = ImageDataset(df=train, img_dir='train/', transform=transform)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=transform)

In [34]:
from torch.utils.data import DataLoader

loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, shuffle=False)

In [35]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        
        self.max_pool = nn.MaxPool2d(kernel_size=2)
        self.avg_pool = nn.AvgPool2d(kernel_size=2)
        
        self.fc = nn.Linear(in_features= 64 * 4 * 4, out_features=2)
    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.max_pool(F.relu(self.conv2(x)))
        x = self.avg_pool(x)
        x = x.view(-1, 64 * 4 * 4) # 평탄화
        x = self.fc(x)
        return x
        

In [36]:
model = Model().to(device)
model

Model(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=2, bias=True)
)

In [37]:
criterion = nn.CrossEntropyLoss()

In [38]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [40]:
epochs = 10

for epoch in range(epochs):
    epoch_loss = 0 # loss 초기화
    
    for images, labels in loader_train:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad() # 기울기 초기화
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        # 손실 추가
        epoch_loss += loss.item()
        
        # UPDATE
        loss.backward()
        optimizer.step()
    print(f"에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_loss/len(loader_train):.4f}")

에폭 [1/10] - 손실값: 0.1581
에폭 [2/10] - 손실값: 0.1499
에폭 [3/10] - 손실값: 0.1424
에폭 [4/10] - 손실값: 0.1341
에폭 [5/10] - 손실값: 0.1306
에폭 [6/10] - 손실값: 0.1238
에폭 [7/10] - 손실값: 0.1172
에폭 [8/10] - 손실값: 0.1154
에폭 [9/10] - 손실값: 0.1095
에폭 [10/10] - 손실값: 0.1081


In [41]:
from sklearn.metrics import roc_auc_score

true_list = []
preds_list = []

In [43]:
model.eval() # 모델을 평가 상태로 설정

with torch.no_grad(): # 기울기 계산 비활성화
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:,1]
        true = labels.cpu()
        
        preds_list.extend(preds)
        true_list.extend(true)
        
print(f"검증 데이터의 ROC AUC: {roc_auc_score(true_list, preds_list):.4f}")

검증 데이터의 ROC AUC: 0.9923


In [44]:
dataset_test = ImageDataset(df=submission, img_dir="test/", transform=transform)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, shuffle=True)

In [46]:
model.eval() # 모델을 평가 상태로 설정

preds = []

with torch.no_grad(): # 기울기 계산 비활성화
    for images, labels in loader_test:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        preds_part = torch.softmax(outputs.cpu(), dim=1)[:,1].tolist()
        
        preds.extend(preds_part)

In [47]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index = False)

In [48]:
import shutil

shutil.rmtree('./train')
shutil.rmtree('./test')